In [6]:
import numpy as np
import torch
from math import sqrt
train_data = np.random.rand(2,100)
train_label = np.where(train_data[0] * train_data[1] < 0.25, -1, 1)
def Frax(n):
    axis = n / torch.norm(n)
    gate = torch.zeros((2, 2), dtype=torch.complex64)
    gate[0, 0], gate[0, 1], gate[1, 0], gate[1, 1] = -axis[2]*1j, -axis[0]*1j-axis[1], -axis[0]*1j+axis[1], axis[2]*1j
    return gate
def amplitude_embedding(data1, data2): 
    ans = torch.tensor([data1,data2]).to(torch.complex64)
    return ans / torch.norm(ans)
X = torch.tensor([[0,1],[1,0]], dtype=torch.complex64)
Y = torch.tensor([[0,-1j],[1j,0]], dtype=torch.complex64)
Z = torch.tensor([[1,0],[0,-1]], dtype=torch.complex64)
XY = (X+Y)/sqrt(2)
XZ = (X+Z)/sqrt(2)
YZ = (Y+Z)/sqrt(2)
def lastbit_Z(state):
    return torch.sum(state[0:len(state):2].abs()**2)-torch.sum(state[1:len(state):2].abs()**2)
def predict(n1, n2, write=False):
    predict = []
    acc = 0
    score = 0
    for i in range(100):
        data = amplitude_embedding(train_data[0,i], train_data[1,i])
        predict.append(lastbit_Z(Frax(n2) @ Frax(n1) @ data))
        score += train_label[i] * predict[-1]
        if predict[-1] * train_label[i] > 0:
            acc += 1
    if write:
        print('Accuracy : ', acc / 100)
        print('Score : ', score)
        print(n1)
        print(n2)
    return score

In [16]:
from os import writev
def optimize(which, n1, n2, write=False):
    R = torch.zeros((3,3))
    for i in range(100):
        x = amplitude_embedding(train_data[0,i], train_data[1,i])
        if which == 'n1':
            rx = Frax(n2) @ X @ x
            ry = Frax(n2) @ Y @ x
            rz = Frax(n2) @ Z @ x
            rxy = Frax(n2) @ XY @ x
            rxz = Frax(n2) @ XZ @ x
            ryz = Frax(n2) @ YZ @ x
        else:
            rx = X @ Frax(n1) @ x
            ry = Y @ Frax(n1) @ x
            rz = Z @ Frax(n1) @ x
            rxy = XY @ Frax(n1) @ x
            rxz = XZ @ Frax(n1) @ x
            ryz = YZ @ Frax(n1) @ x
        rx = lastbit_Z(rx)
        ry = lastbit_Z(ry)
        rz = lastbit_Z(rz)
        rxy = lastbit_Z(rxy)
        rxz = lastbit_Z(rxz)
        ryz = lastbit_Z(ryz)
        R[0,0] += train_label[i] * (2 * rx)
        R[0,1] += train_label[i] * (2 * rxy - rx - ry)
        R[0,2] += train_label[i] * (2 * rxz - rx - rz)
        R[1,1] += train_label[i] * (2 * ry)
        R[1,2] += train_label[i] * (2 * ryz - ry - rz)
        R[2,2] += train_label[i] * (2 * rz)
    R[1,0] = R[0,1]
    R[2,0] = R[0,2]
    R[2,1] = R[1,2]
    eigenvalues, eigenvectors = torch.linalg.eigh(R)
    if which == 'n1':
        predict(eigenvectors[0], n2, write=write)
        predict(eigenvectors[1], n2, write=write)
        predict(eigenvectors[2], n2, write=write)
    else:
        predict(n1, eigenvectors[0], write=write)
        predict(n1, eigenvectors[1], write=write)
        predict(n1, eigenvectors[2], write=write)

In [17]:
n1 = torch.zeros(3)+1/sqrt(3)
n2 = torch.zeros(3)+1/sqrt(3)

score_base = predict(n1, n2, write=True)

Accuracy :  0.5
Score :  tensor(-4.1490)
tensor([0.5774, 0.5774, 0.5774])
tensor([0.5774, 0.5774, 0.5774])


In [18]:
optimize('n1', n1, n2, write=True)

Accuracy :  0.47
Score :  tensor(2.7363)
tensor([-0.3187, -0.4478,  0.8354])
tensor([0.5774, 0.5774, 0.5774])
Accuracy :  0.54
Score :  tensor(-0.5931)
tensor([9.3417e-01, 7.3528e-04, 3.5682e-01])
tensor([0.5774, 0.5774, 0.5774])
Accuracy :  0.37
Score :  tensor(-9.0491)
tensor([ 0.1604, -0.8941, -0.4181])
tensor([0.5774, 0.5774, 0.5774])


In [19]:
M = 20
nrand = torch.rand(M, 3)
max_score = score_base
for i in range(M):
    score = predict(nrand[i], n2, write=False)
    if max_score < score:
        max_score = score
print(max_score)

tensor(5.0543)
